# Tutorial 2: more advanced optical systems and ray-tracing options.
In the previous tutorial we built a simple optical system consisting of a paraboloid. We then performed a ray-trace from the initial frame of rays to the paraboloid surface, and from the paraboloid we found the focus of the frame by calling the s.findRTfocus method. We did all of this using the tubular ray-trace frame input.

In this tutorial, we will introduce Gaussian ray-trace frames. We will then create an optical setup that is slightly more advanced, where we try to generate a collimated beam from a Gaussian ray-trace beam.   

In [ ]:
%matplotlib notebook

import numpy as np

from src.PyPO.System import System

s = System()

In [ ]:
RTpar = {
        "name"      : "start",
        "nRays"     : 5000,
        "lam"       : 0.01,
        "n"         : 1,
        "seed"      : 0,
        "x0"        : 0.05,
        "y0"        : 0.05
        }

s.createGRTFrame(RTpar)
s.translateGrids("start", np.array([0, 0, 100]), obj="frame")
s.plotRTframe("start", project='xy')

We start by defining a Gaussian ray-trace frame. These are created by sampling a Gaussian distribution of positions and directions for the rays. In the dictionary, we specify the number of rays in the frame, the wavelength of the light in millimeters, the refractive index of the medium, the seed for the sampling and the half-power beamwidths along the x and y-axis. This frame is then translated by 100 millimeters along the z-axis.

In [ ]:
ellipse = {
            "name"      : "ellipsoid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "flip"      : True,
            "focus_1"   : np.array([0, 0, 100]),
            "focus_2"   : np.array([0, 0, -100]),
            "orient"    : "z",
            "ecc"       : 0.5,
            "lims_u"    : np.array([0, 30]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([801, 801])
            }

s.addEllipse(ellipse)

plane_t = {
            "name"      : "plane_t",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 10]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([801, 801])
            }

s.addPlane(plane_t)
s.rotateGrids("plane_t", np.array([45, 0, 0]))

parabola = {
            "name"      : "paraboloid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "focus_1"   : np.array([0, -100, 0]),
            "vertex"    : np.array([-10, -100, 0]),
            "lims_u"    : np.array([0, 5]),
            "lims_v"    : np.array([0, 360]),
            "gcenter"   : np.array([0,-20]),
            "gridsize"  : np.array([801, 801])
            }

s.addParabola(parabola)

s.plot3D("paraboloid")

plane_out = {
            "name"      : "plane_out",
            "gmode"     : "xy",
            "lims_x"    : np.array([-10, 10]),
            "lims_y"    : np.array([-10, 10]),
            "gridsize"  : np.array([801, 801])
            }

s.addPlane(plane_out)
s.rotateGrids("plane_out", np.array([0, -90, 0]))
s.translateGrids("plane_out", np.array([50, -120, 0]))

s.plotSystem()

We added several optical elements to the setup. First, we added an ellipsoid reflector. The focii are placed at z=100 and z=-100. Because we translated the Gaussian frame earlier, the frame is now situated in the upper focus of the ellipsoid. The semi-major of the ellipsoid is oriented along the z-axis by setting 'orient' to 'z'. 

The second element, 'plane_t', is a tilted plane. This plane is situated at z=0 and is tilted by 45 degrees around the x-axis. This mirror essentially folds the lower focus of the ellipsoid to lie along the negative y-axis.

After the tilted plane, we place an off-axis paraboloid section. It is generated by adjusting the position xy surface generated by the uv parametrisation using the 'gcenter' parameter, which contains the x and y-offset, respectively. Because the paraboloid is generated in its own restframe first and then transformed to its position in the system, we can use the xy center option to select off-axis sections.
The focus of the paraboloid is placed in the lower ellipse focus, after being folded by the tilted plane.

The final element is a plane placed after the paraboloid, along the x-axis. We terminate the ray-trace in this plane and inspect the results.

We can inspect the designed system by calling the plotSystem method of the system. This plots all elements in the current system object.

In [ ]:
start_ell_RT = {
            "fr_in"     : "start",
            "t_name"    : "ellipsoid",
            "fr_out"    : "fr_ell",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(start_ell_RT)

ell_plane_t_RT = {
            "fr_in"     : "fr_ell",
            "t_name"    : "plane_t",
            "fr_out"    : "fr_plane_t",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(ell_plane_t_RT)

plane_t_par_RT = {
            "fr_in"     : "fr_plane_t",
            "t_name"    : "paraboloid",
            "fr_out"    : "fr_par",
            "device"    : "CPU",
            "t0"        : 120,
            "tol"       : 1e-6
            }

s.runRayTracer(plane_t_par_RT)

par_plane_out_RT = {
            "fr_in"     : "fr_par",
            "t_name"    : "plane_out",
            "fr_out"    : "fr_out",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(par_plane_out_RT)
s.findRTfocus("fr_plane_t")
s.plotSystem(RTframes=["start", "fr_ell", "fr_plane_t", "fr_par", "fr_out"])
s.plotRTframe("fr_out", project='yz')

The propagations are similar to the one in the previous tutorial, albeit a bit more plentiful. It is good to note that for the propagation from the tilted plane to the paraboloid, we specify the 't0' parameter. This is the initial guess for the ray-tracer. Because we select, in this case, the section from the half of the paraboloid that is furthest away from the tilted plane, the ray-tracer will first encounter the closer half of the paraboloid. This is more easily imagined when we remember that, after transforming the paraboloid and incoming frame into the parabooid restframe, the frame is essentially illuminating the full paraboloid from the side. If we did not change t0, the ray-tracer would instead intersect in the closest half of the paraboloid (to see this, just set t0 to 1 and repeat the propagation).

To plot the rays and optical elements together, we call plotSystem again, but now passing the 'RTframes' argument. This is a list containing the frames to be plotted. The frames are connected in the order in which they are passed to plotSystem.